In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
%cd '/content/drive/My Drive/'

Mounted at /content/drive
/content/drive/My Drive


In [3]:
!pip install leidenalg igraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import scipy.io as sio
import numpy as np
import igraph as ig
import leidenalg
from itertools import combinations
mat=sio.loadmat('A_.mat')['A'] #Import the multiplex network of sparse kernel matrices
mat

array([[[0.99703998, 0.99980489, 0.99836291, 1.        , 1.        ,
         1.        ],
        [0.99995421, 0.9999995 , 0.99711599, 0.        , 0.99772926,
         0.99982407],
        [0.99968319, 0.99972965, 0.99498725, 0.99999293, 0.99479397,
         0.        ],
        ...,
        [0.99998747, 0.99982494, 0.99830278, 0.99999919, 0.99664331,
         0.99999946],
        [0.99994689, 0.99972221, 0.99857382, 0.        , 0.99619615,
         0.99999271],
        [0.97722456, 0.99909776, 0.9994047 , 0.        , 0.99056986,
         0.99974162]],

       [[0.99900075, 0.9948156 , 0.99994813, 0.        , 0.99772926,
         0.99982407],
        [0.99373151, 0.99274004, 0.99996325, 1.        , 1.        ,
         1.        ],
        [0.98926986, 0.        , 0.99946143, 0.        , 0.99939661,
         0.99970599],
        ...,
        [0.99200317, 0.99470929, 0.99995821, 0.        , 0.99989379,
         0.99984309],
        [0.99132308, 0.99519562, 0.99990197, 0.        , 0.999

In [5]:
def Divisive_Clustering(X):
    """
    Divisive Kernel Clustering using the Leiden algorithm.

    Arguments:
    X : 3D numpy array of shape (N, N, L), where N is the number of nodes,
        and L is the number of layers (each representing a weighted adjacency matrix).

    Returns:
    Opt_rank : The optimal number of clusters to extract
    M : Community assignment of nodes
    Q : Modularity value of the partition
    aggregated_vs : Co-membership matrix aggregated across layers
    """
    # Validate input dimensions
    if len(X.shape) != 3 or X.shape[0] != X.shape[1]:
        raise ValueError("Input X must be a 3D numpy array with shape (N, N, L).")

    N, _, L = X.shape  # Number of nodes and layers
    Vs = []  # To store co-membership matrices from each layer
    combos_s = list(combinations(range(N), 2))  # Generate pairs of nodes

    for i in range(L):
        v = []  # Co-membership vector for the current layer
        net = X[:, :, i]  # Extract the i-th layer as a NumPy array

        # Create an igraph directed graph for each layer
        G = ig.Graph.Weighted_Adjacency(net.tolist(), mode="directed", attr="weight")
        partition = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition, weights="weight")

        M = partition.membership

        # Compute co-membership matrix for the current layer
        co_matrix = np.zeros((N, N))
        for ii in range(len(combos_s)):
            m1 = M[combos_s[ii][0]]
            m2 = M[combos_s[ii][1]]
            if m1 == m2:
                co_matrix[combos_s[ii][0], combos_s[ii][1]] = net[combos_s[ii][0], combos_s[ii][1]]+net[combos_s[ii][1], combos_s[ii][0]]
                co_matrix[combos_s[ii][1], combos_s[ii][0]] = net[combos_s[ii][0], combos_s[ii][1]]+net[combos_s[ii][1], combos_s[ii][0]]  # Ensure symmetry

        Vs.append(co_matrix)

    # Convert the list of co-membership matrices to a 3D array
    Vs = np.stack(Vs, axis=2)

    # Aggregate the co-membership matrices by summing across layers
    aggregated_vs = np.sum(Vs, axis=2)

    # Create a graph for the aggregated matrix (using undirected mode)
    # Aggregated co-membership matrix should be treated as undirected
    G = ig.Graph.Weighted_Adjacency(aggregated_vs.tolist(), mode="undirected", attr="weight")
    partition = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition, weights="weight")

    Q = partition.q  # Modularity score
    M = partition.membership

    # Return the optimal number of clusters and other results
    Opt_rank = max(M) + 1
    return Opt_rank, M, aggregated_vs



In [6]:
Opt_rank, M, aggregated_vs= Divisive_Clustering(mat)
print("Optimal number of clusters:", Opt_rank)
print("Community assignments:", M)
print("Aggregated co-membership matrix (Vs):\n", aggregated_vs)

Optimal number of clusters: 3
Community assignments: [1, 2, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 2, 0, 0, 1, 2, 2, 1, 0, 1, 1, 1, 0, 2, 0, 1, 0, 0, 1, 2, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1, 2, 2, 2, 1]
Aggregated co-membership matrix (Vs):
 [[0.         0.         5.99659234 ... 5.9969265  1.99998543 5.96121352]
 [0.         0.         5.99732425 ... 5.9944345  3.99944306 4.98140943]
 [5.99659234 5.99732425 0.         ... 7.99811488 3.99850776 0.        ]
 ...
 [5.9969265  5.9944345  7.99811488 ... 0.         5.99911907 3.98954385]
 [1.99998543 3.99944306 3.99850776 ... 5.99911907 0.         1.99964221]
 [5.96121352 4.98140943 0.         ... 3.98954385 1.99964221 0.        ]]


In [7]:
def recursive_divisive_clustering(X):
    """
    Recursively applies a given Divisive_Clustering function to the input tensor X,
    dividing into subgroups based on the communities identified until no further division is possible.

    Arguments:
    X : 3D numpy array of shape (N, N, L), where N is the number of nodes,
        and L is the number of layers (each representing a weighted adjacency matrix).
    divisive_clustering_function : Function to perform divisive clustering.

    Returns:
    result_tree : A nested dictionary representing the hierarchical clustering structure.
    """
    def recursive_helper(X, level=0):
        """
        Helper function to recursively apply clustering and store the results.
        """
        # Apply the provided divisive clustering function
        Opt_rank, M, aggregated_vs = Divisive_Clustering(X)

        # Prepare the results dictionary
        result_tree = {
            "Level": level,
            "Opt_rank": Opt_rank,
            "Membership": M,
            "Subclusters": {}
        }

        # Stop recursion if there is only one community
        if Opt_rank <= 1:
            return result_tree

        # Recursively process each cluster
        unique_clusters = np.unique(M)
        for cluster in unique_clusters:
            nodes_in_cluster = np.where(np.array(M) == cluster)[0]

            # Extract the subgraph for the current cluster
            subgraph = X[np.ix_(nodes_in_cluster, nodes_in_cluster, range(X.shape[2]))]

            # Apply the recursive helper to the subgraph
            result_tree["Subclusters"][cluster] = recursive_helper(subgraph, level + 1)

        return result_tree

    # Start the recursion
    return recursive_helper(X)

# Example usage:
# result = recursive_divisive_clustering(X, Divisive_Clustering)


In [8]:
result = recursive_divisive_clustering(mat)
result

{'Level': 0,
 'Opt_rank': 2,
 'Membership': [0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0],
 'Subclusters': {0: {'Level': 1,
   'Opt_rank': 2,
   'Membership': [1,
    0,
    1,
    1,
    1,
    1,
    1,
    0,
    0,
    1,
    0,
    0,
    0,
    0,
    0,
    1,
    0,
    0,
    1,
    1,
    0,
    1,
    1,
    0,
    0,
    0,
    1],
   'Subclusters': {0: {'Level': 2,
     'Opt_rank': 3,
     'Membership': [2, 1, 0, 1, 1, 2, 2, 0, 1, 0, 0, 2, 0, 0],
     'Subclusters': {0: {'Level': 3,
       'Opt_rank': 2,
       'Membership': [1, 0, 0, 0, 1, 1],
       'Subclusters': {0: {'Level': 4,
         'Opt_rank': 1,
         'Membership': [0, 0, 0],
         'Subclusters': {}},
        1: {'Level': 4,
         'Opt_rank': 1,
         'Membership': [0, 0, 0],
         'Subclusters': {